# Source Data from PA

In [66]:
from TM1py.Services import TM1Service
from TM1py.Utils.Utils import element_names_from_element_unique_names, build_pandas_dataframe_from_cellset
from creds import tm1_credentials

with TM1Service(address=tm1_credentials["address"], port=tm1_credentials["port"], user=tm1_credentials["user"], password=tm1_credentials["password"], ssl=tm1_credentials["ssl"]) as tm1:
    data = tm1.cubes.cells.execute_view(cube_name='}ElementAttributes_DMD Transmission Route', view_name='Default', private=False)
    distributionlocation = build_pandas_dataframe_from_cellset(data, multiindex=False)
    data = tm1.cubes.cells.execute_view(cube_name='}ElementAttributes_Customer', view_name='Default', private=False)
    customerlocation = build_pandas_dataframe_from_cellset(data, multiindex=False)

In [67]:
distributionlocation.set_index('DMD Transmission Route', inplace=True)
customerlocation.set_index('Customer', inplace=True)

# Geocoding and Distance Calculation

In [68]:
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
geolocator = Nominatim(user_agent='Western')

In [ ]:
coords = []
for distribution_location in distributionlocation.index:
    location = None
    while location is None:
        try:
            location = geolocator.geocode(distributionlocation.loc[distribution_location]['Values'])
        except:
            pass

    coords.append((location.latitude, location.longitude))
distributionlocation['Coordinates'] = coords

In [ ]:
len(coords)

In [ ]:
coords = []
for customer_location in customerlocation.index:
    location = geolocator.geocode(customerlocation.loc[customer_location]['Values'])
    coords.append((location.latitude, location.longitude))
customerlocation['Coordinates'] = coords

# Push Data Back to PA

In [59]:
customercellset = {(key, 'Coordinates'): row['Coordinates'] for key, row in customerlocation.iterrows()}
distributioncellset = {(key, 'Coordinates'): row['Coordinates'] for key, row in distributionlocation.iterrows()}

KeyError: 'Coordinates'

In [60]:
with TM1Service(address=tm1_credentials["address"], port=tm1_credentials["port"], user=tm1_credentials["user"], password=tm1_credentials["password"], ssl=tm1_credentials["ssl"]) as tm1:
    tm1.cubes.cells.write_values('}ElementAttributes_Customer', customercellset, dimensions=['Customer', '}ElementAttributes_Customer'])
    tm1.cubes.cells.write_values('}ElementAttributes_DMD Transmission Route', distributioncellset, dimensions=['DMD Transmission Route', '}ElementAttributes_DMD Transmission Route'])